# Exercises Deep Learning
First Lecture

## Basic Tensor Operations


In [12]:
import torch
import numpy as np
from torch import nn

Different ways to create tensors:
- ```torch.zeros```: Creates a tensor filled with zeros
- ```torch.ones```: Creates a tensor filled with ones
- ```torch.rand```: Creates a tensor with random values uniformly sampled between 0 and 1
- ```torch.randn```: Creates a tensor with random values sampled from a normal distribution with mean 0 and variance 1
- ```torch.arange```: Creates a tensor containing the values
- ```torch.Tensor``` (input list): Creates a tensor from the list elements you provide

You can obtain the shape of a tensor in the same way as in numpy (```x.shape```), or using the ```.size``` method:

In [13]:
x = torch.Tensor(2, 3, 4)

In [15]:
shape = x.shape
print("Shape:", x.shape)

size = x.size()
print("Size:", size)

Shape: torch.Size([2, 3, 4])
Size: torch.Size([2, 3, 4])


Tensor to Numpy, and Numpy to Tensor


In [ ]:
np_arr = np.array([[1, 2], [3, 4]])
tensor = torch.from_numpy(np_arr)

print("Numpy array:", np_arr)
print("PyTorch tensor:", tensor)

In [ ]:
tensor = torch.arange(4)
np_arr = tensor.numpy()

print("PyTorch tensor:", tensor)
print("Numpy array:", np_arr)

Matrix multiplication

In [ ]:
x = torch.arange(6)
x = x.view(2, 3)
print("X", x)

In [ ]:
W = torch.arange(9).view(3, 3) # We can also stack multiple operations in a single line
print("W", W)

In [ ]:
h = torch.matmul(x, W) # Verify the result by calculating it by hand too!
print("h", h)

 ### What about gpus?

When you create a tensor the tensor is ready to be computed by the cpu. To convert the tensor you can use ```.to()```
passing to the function "cuda" or "cpu" as needed

#### How do I know if I have cuda cores on my computer?
To solve this you can check with torch if cuda is available:

In [ ]:
example_tensor = torch.rand(2,2)
if torch.cuda.is_available():
    print("CUDA is available. You can use GPU for PyTorch.")
    example_tensor.to("cuda")
else:
    print("CUDA is not available. Using CPU for PyTorch.")
    example_tensor.to("cpu")

### Exercises

#### 1. Create two tensors

   - A 3x3 tensor of random numbers.
   - A 3x3 tensor filled with ones.

In [24]:
a = torch.rand(3, 3)
b = torch.ones(3, 3)

#### 2. Perform the following operations

- Add the two tensors.
- Multiply the two tensors element-wise.
- Compute the dot product between the first row of both tensors.
 - Find the transpose of the resulting tensor from the element-wise multiplication.

In [25]:
c = a + b
d = a * b
e = torch.dot(a[0], b[0])
f = b.t()

#### 3. Convert the resulting tensor to a NumPy array and back to a PyTorch tensor.

In [ ]:
torch_tensor = torch.tensor([[1, 2], [3, 4]])
numpy_array = torch_tensor.numpy()
print(torch_tensor)
print(numpy_array)

In [3]:
torch_tensor_back = torch.from_numpy(numpy_array)
print(torch_tensor_back)

tensor([[1, 2],
        [3, 4]])


## Autograd
```torch.autograd``` is PyTorch’s automatic differentiation engine that powers neural network training.


1. Create Tensors

In [4]:
x_a = torch.tensor(0., requires_grad=True)
x_b = torch.tensor(0., requires_grad=True)
w_a = torch.tensor(0.9, requires_grad=True)
w_b = torch.tensor(0.9, requires_grad=True)

y = torch.tensor(0., requires_grad=False)

2. Build a computation graph

In [5]:
weighted_a = w_a * x_a
weighted_b = w_b * x_b
sum_unit = weighted_a + weighted_b

3. Activation Function

For a simple approach as ease of replication by hand we will this activation function:

In [6]:
y_hat = torch.sigmoid(sum_unit)

4. Calculate Loss

In [7]:
loss = torch.nn.BCELoss()
output = loss(y_hat, y)


5. Calculate gradients

In [8]:
output.backward()

6.Print out the gradients

In [9]:
print(x_a.grad)
print(x_b.grad)
print(w_a.grad)
print(w_b.grad)

tensor(0.4500)
tensor(0.4500)
tensor(0.)
tensor(0.)


### Training Loop

In [10]:
learning_rate = 0.1
epochs = 100

input_data = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32)
target_data = torch.tensor([[0], [0], [0], [1]], dtype=torch.float32)

In [16]:
class ANDGateModel(nn.Module):
    def __init__(self):
        super(ANDGateModel, self).__init__()
        self.linear = nn.Linear(2, 1,bias=True)

    def forward(self, x):
        x = self.linear(x)
        x = torch.sigmoid(x)
        return x


In [17]:
# Initialize the model
model = ANDGateModel()

# Loss function (Binary Cross-Entropy Loss)
loss_fn = torch.nn.BCELoss()

# Optimizer (Stochastic Gradient Descent)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    y_hat = model(input_data)
    loss = loss_fn(y_hat, target_data)


    loss.backward() # Backpropagation
    optimizer.step() # Update parameters using the optimizer
    optimizer.zero_grad() # Zero the gradients for the next iteration

    # Print loss and progress every 1000 epochs
    if (epoch + 1) % 1000 == 0:
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.4f}")

# Final weights and bias (optional)
print(f"Final weights: {model.linear.weight.data}")
print(f"Final bias: {model.linear.bias.data}")

# Test the AND gate
with torch.no_grad():
    for i in range(len(input_data)):
        x_a, x_b = input_data[i]
        y_hat = model(torch.tensor([[x_a, x_b]]))  # Model expects a batch
        print(f"Input: {input_data[i].numpy()} -> Predicted Output: {round(y_hat.item())}, Raw Output: {y_hat.item():.4f}")


Final weights: tensor([[3.6639, 3.6537]])
Final bias: tensor([-5.4969])
Input: [0. 0.] -> Predicted Output: 0, Raw Output: 0.0041
Input: [0. 1.] -> Predicted Output: 0, Raw Output: 0.1367
Input: [1. 0.] -> Predicted Output: 0, Raw Output: 0.1379
Input: [1. 1.] -> Predicted Output: 1, Raw Output: 0.8606


### Exercises

#### 1.Replicate the OR Gate using a Neural Network
 Objective:
- Train a neural network to approximate the function of an OR gate.
- Compare how changing the weights or biases impacts the output of the network.

Input 1 | Input 2 | Output (OR)
| -- | -- | --|
0 | 0 | 0
0 | 1 | 1
1 | 0 | 1
1 | 1 | 1

1. Create the dataset
2. Replicate the architecture from the AND gate example
3. Change the loss function from Binary Cross-Entropy to Mean Squared Error

In [ ]:
# Code Here

In [ ]:
# https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html

#### 2. Build and train a network
1. Build a simple fully connected neural network with the following architecture:
    - Input layer with 2 units
    - Hidden layer with 4 units and ReLU activation
    - Output layer with 1 unit
2. Define the following loss function and optimizer:
    - Loss: Mean Squared Error (MSE)
    - Optimizer: Stochastic Gradient Descent (SGD)

In [27]:
import torch
import torch.nn as nn

# Define the neural network
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        # Define layers here
        self.layer1 = nn.Linear(2, 4)
        self.activation_function = nn.ReLU()
        self.layer2 =   nn.Linear(4, 1)

    def forward(self, x):
        # Define forward pass
        x = self.layer1(x)
        x = self.activation_function(x)
        x = self.layer2(x)
        return x

# Create synthetic data
x_train = torch.rand(100, 2)
y_train = 2 * x_train[:, 0] + 3 * x_train[:, 1]
y_train = y_train.view(-1, 1)

# Initialize the model, loss function, and optimizer
model = SimpleNet()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Training loop
for epoch in range(100):
    model.train()

    # Forward pass
    y_pred = model(x_train)

    # Compute loss
    loss = criterion(y_pred, y_train)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Final model predictions
print("Final Predictions:", model(x_train[:5]))
print("Actual Values:", y_train[:5])


Epoch 10, Loss: 5.695849418640137
Epoch 20, Loss: 2.8809168338775635
Epoch 30, Loss: 1.3397232294082642
Epoch 40, Loss: 0.8157290816307068
Epoch 50, Loss: 0.6970154643058777
Epoch 60, Loss: 0.6625199913978577
Epoch 70, Loss: 0.6401515007019043
Epoch 80, Loss: 0.6194170117378235
Epoch 90, Loss: 0.5990427136421204
Epoch 100, Loss: 0.5788787007331848
Final Predictions: tensor([[2.2517],
        [2.8239],
        [2.6435],
        [2.6913],
        [2.9278]], grad_fn=<AddmmBackward0>)
Actual Values: tensor([[1.1961],
        [2.3344],
        [2.5772],
        [2.6616],
        [3.3427]])
